In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler

import time


In [20]:
# !pip install xgboost

In [21]:
from xgboost import XGBRanker

In [22]:
df = pd.read_csv('train_preprocessed_2.csv')
df.head()

,srch_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,...,day_of_week,relevance,prop_starrating_location_interaction,price_historical_interaction,total_time_duration,highly_rated_prop,total_people,mean_price_by_country,has_promotion_and_highly_rated,normalized_price
0,1,219,893,3,3.5,1,2.83,0.0438,4.95,27,...,3,0.0,8.49,3.952173,1,0,4,0.000941,0,0.997827
1,1,219,10404,4,4.0,1,2.20,0.0149,5.03,26,...,3,0.0,8.80,4.033542,1,1,4,0.000428,0,0.996458
2,1,219,21315,3,4.5,1,2.20,0.0245,4.92,21,...,3,0.0,6.60,3.923730,1,0,4,0.000616,0,0.996270
3,1,219,27348,2,4.0,1,2.83,0.0125,4.39,34,...,3,0.0,5.66,3.402506,1,0,4,0.009392,0,0.987494
4,1,219,29604,4,3.5,1,2.64,0.1241,4.93,4,...,3,0.0,10.56,3.932978,1,1,4,0.000136,0,0.997022


In [23]:
df.columns

Index(['srch_id', 'prop_country_id', 'prop_id', 'prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1',
       'prop_location_score2', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool',
       'click_bool', 'booking_bool', 'month', 'day_of_week', 'relevance',
       'prop_starrating_location_interaction', 'price_historical_interaction',
       'total_time_duration', 'highly_rated_prop', 'total_people',
       'mean_price_by_country', 'has_promotion_and_highly_rated',
       'normalized_price'],
      dtype='object')

In [24]:
X = df
train_groups = X.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

relevances = X['relevance']

X= X.loc[:, ~X.columns.isin(['srch_id','relevance'])]
y = relevances

In [25]:
X.columns

Index(['prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'random_bool', 'click_bool',
       'booking_bool', 'month', 'day_of_week',
       'prop_starrating_location_interaction', 'price_historical_interaction',
       'total_time_duration', 'highly_rated_prop', 'total_people',
       'mean_price_by_country', 'has_promotion_and_highly_rated',
       'normalized_price'],
      dtype='object')

In [26]:
X = X.drop(['booking_bool','position','random_bool','prop_id','click_bool'], axis=1)


In [ ]:
model = XGBRanker(
                  objective='rank:ndcg', 
                  n_estimators=500, 
                  random_state=0,
                  learning_rate=0.1)


model.fit(
    X,
    y,
    group=train_groups,
    eval_metric='ndcg@5',
    eval_set=[(X, y)],
    eval_group=[list(train_groups)],
    verbose=True,
    early_stopping_rounds=10
)

/Users/freekcool/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/freekcool/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-ndcg@5:0.97590
[1]	validation_0-ndcg@5:0.97664
[2]	validation_0-ndcg@5:0.97691
[3]	validation_0-ndcg@5:0.97691
[4]	validation_0-ndcg@5:0.97692
[5]	validation_0-ndcg@5:0.97689
[6]	validation_0-ndcg@5:0.97692
[7]	validation_0-ndcg@5:0.97696
[8]	validation_0-ndcg@5:0.97699
[9]	validation_0-ndcg@5:0.97703
[10]	validation_0-ndcg@5:0.97705
[11]	validation_0-ndcg@5:0.97711
[12]	validation_0-ndcg@5:0.97715
[13]	validation_0-ndcg@5:0.97719
[14]	validation_0-ndcg@5:0.97724
[15]	validation_0-ndcg@5:0.97724
[16]	validation_0-ndcg@5:0.97730
[17]	validation_0-ndcg@5:0.97734
[18]	validation_0-ndcg@5:0.97738
[19]	validation_0-ndcg@5:0.97746
[20]	validation_0-ndcg@5:0.97750
[21]	validation_0-ndcg@5:0.97751
[22]	validation_0-ndcg@5:0.97757
[23]	validation_0-ndcg@5:0.97764
[24]	validation_0-ndcg@5:0.97795
[25]	validation_0-ndcg@5:0.97805
[26]	validation_0-ndcg@5:0.97814
[27]	validation_0-ndcg@5:0.97822
[28]	validation_0-ndcg@5:0.97826
[29]	validation_0-ndcg@5:0.97829
[30]	validation_0-nd

In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')

In [ ]:
test_df = pd.read_csv('test_preprocessed_2.csv')

In [26]:
print(len(test_df))

4959183


In [27]:
def sort_properties(srch_id, scores, prop_ids):
    sorted_group = pd.DataFrame(columns=['srch_id', 'prop_id', 'scores'])
    sorted_group['srch_id'] = len(scores)*[srch_id]
    sorted_group['prop_id'] = prop_ids.values
    sorted_group['scores'] = scores
    sorted_group = sorted_group.sort_values(by='scores', ascending=False)
    return sorted_group 

def recommend_properties(model, data):
    result = pd.DataFrame(columns=['srch_id', 'prop_id', 'scores'])
    df_list = []
    grouped_data = data.groupby('srch_id')
    for group_name, group in tqdm(grouped_data):
        scores = model.predict(group.loc[:, ~group.columns.isin(['srch_id','prop_id'])])
        sorted_group = sort_properties(group_name, scores, group['prop_id'])
        df_list.append(sorted_group)
    result = pd.concat(df_list)
    return result

In [28]:
X.columns

Index(['prop_country_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'month', 'day_of_week',
       'highly_rated_prop', 'total_people'],
      dtype='object')

In [29]:
test_df.columns

Index(['srch_id', 'prop_country_id', 'prop_id', 'prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1',
       'prop_location_score2', 'prop_log_historical_price', 'price_usd',
       'promotion_flag', 'srch_destination_id', 'srch_length_of_stay',
       'srch_booking_window', 'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'month', 'day_of_week',
       'highly_rated_prop', 'total_people'],
      dtype='object')

In [30]:
start = time.time()
recommendations = recommend_properties(model, test_df)
end = time.time()
print(f'Total Time for validation prediction:{end - start}')
submission = recommendations.drop(columns=['scores'])
submission.to_csv(f'xgbranker_predictions.csv', index=False)

100%|██████████| 199549/199549 [13:26<00:00, 247.31it/s]
